In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bertviz

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import pickle
import numpy as np

from tqdm import trange

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyl

# **Open the data**

The data has been already split into train and test:

In [ ]:
#f_in = open("drive/MyDrive/data/15-classes/train_set_paragraphs.pkl","rb")
#f_in = open("drive/MyDrive/data/15-classes/train_set_paragraphs_augm-113.pkl","rb")
#f_in = open("drive/MyDrive/data/15-classes/train_set_sentences.pkl","rb")
f_in = open("drive/MyDrive/data/15-classes/train_set_sentences_augm-105.pkl","rb")

train_data = pickle.load(f_in)

f_in.close()

#f_in = open("drive/MyDrive/data/15-classes/test_set_paragraphs.pkl","rb")
f_in = open("drive/MyDrive/data/15-classes/test_set_sentences.pkl","rb")

test_data = pickle.load(f_in)

f_in.close()

# **Training**

In [ ]:
average_results = []

# Fix random seed to make results reproducible:
torch.manual_seed(256)

# Repeat experiment 5 times, report averaged results:
for j in range(5):
    print("\nExperiment",j+1,"--------------------------------------------------------------")
    # Data preprocessing:

    model_name = "bert-base-uncased"

    # Define tokenizer:
    tokenizer = BertTokenizer.from_pretrained(model_name,do_lower_case=True)

    # Tokenize training and test sets:
    tokenizer_train = tokenizer([train_data[i][1] for i in range(len(train_data))], padding="longest", truncation = True, return_tensors="pt")

    tokenizer_test = tokenizer([test_data[i][1] for i in range(len(test_data))], padding="longest", truncation = True, return_tensors="pt")

    # Define Dataloaders:
    batch_size = 16

    train_set = TensorDataset(tokenizer_train['input_ids'],
                              tokenizer_train['attention_mask'],
                              torch.tensor([int(train_data[i][2]) for i in range(len(train_data))]))

    test_set = TensorDataset(tokenizer_test['input_ids'],
                             tokenizer_test['attention_mask'],
                             torch.tensor([int(test_data[i][2]) for i in range(len(test_data))]))

    train_dataloader = DataLoader(
                train_set,
                sampler = RandomSampler(train_set),
                batch_size = batch_size
            )

    validation_dataloader = DataLoader(
                test_set,
                sampler = SequentialSampler(test_set),
                batch_size = batch_size
            )

    # Define the model:
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=15)

    model.to(device)

    # Define parameters and metrics to optimize:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=10e-8)

    def flatten_preds(preds):
      preds = np.argmax(preds, axis = 1).flatten()

      return preds.tolist()

    # Training and validation:
    epochs = 10

    train_loss_set = []
    count = 0
    best_acc_weight = 0
    best_results = []
    # trange is a tqdm wrapper around the normal python range
    for _ in trange(epochs, desc="Epoch"):
        # Tracking variables for training
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0

        # Train the model
        model.train()
        for step, batch in enumerate(train_dataloader):
            # Add batch to device CPU or GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            outputs = model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            # Get loss value
            loss = outputs[0]
            # Add it to train loss list
            train_loss_set.append(loss.item())
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()

            # Update tracking variables
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1

        print("\nTrain loss: {}".format(tr_loss/nb_tr_steps))

        # Tracking variables for validation
        val_preds = []
        val_labels = []
        # Validation of the model
        model.eval()
        # Evaluate data for one epoch
        for batch in validation_dataloader:
            # Add batch to device CPU or GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Telling the model not to compute or store gradients, saving memory and speeding up validation
            with torch.no_grad():
                # Forward pass, calculate logit predictions
                outputs =  model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
                loss, logits = outputs[:2]
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            labels = b_labels.to('cpu').numpy()
            # Update predicted and true labels
            val_preds += flatten_preds(logits)
            val_labels += labels.tolist()

        # Cost-based validation:
        validation_cost = np.zeros(len(val_labels),float)
        validation_cost+=1
        weight_1 = len(val_labels)/float(15)/float(len([i for i in val_labels if i==1]))
        weight_2 = len(val_labels)/float(15)/float(len([i for i in val_labels if i==2]))
        weight_3 = len(val_labels)/float(15)/float(len([i for i in val_labels if i==3]))
        weight_4 = len(val_labels)/float(15)/float(len([i for i in val_labels if i==4]))
        weight_5 = len(val_labels)/float(15)/float(len([i for i in val_labels if i==5]))
        weight_6 = len(val_labels)/float(15)/float(len([i for i in val_labels if i==6]))
        weight_7 = len(val_labels)/float(15)/float(len([i for i in val_labels if i==7]))
        weight_8 = len(val_labels)/float(15)/float(len([i for i in val_labels if i==8]))
        weight_9 = len(val_labels)/float(15)/float(len([i for i in val_labels if i==9]))
        weight_10 = len(val_labels)/float(15)/float(len([i for i in val_labels if i==10]))
        weight_11 = len(val_labels)/float(15)/float(len([i for i in val_labels if i==11]))
        weight_12 = len(val_labels)/float(15)/float(len([i for i in val_labels if i==12]))
        weight_13 = len(val_labels)/float(15)/float(len([i for i in val_labels if i==13]))
        weight_14 = len(val_labels)/float(15)/float(len([i for i in val_labels if i==14]))
        for i in range(len(val_labels)):
            if val_labels[i] == 1:
                validation_cost[i] = weight_1
            elif val_labels[i] == 2:
                validation_cost[i] = weight_2
            elif val_labels[i] == 3:
                validation_cost[i] = weight_3
            elif val_labels[i] == 4:
                validation_cost[i] = weight_4
            elif val_labels[i] == 5:
                validation_cost[i] = weight_5
            elif val_labels[i] == 6:
                validation_cost[i] = weight_6
            elif val_labels[i] == 7:
                validation_cost[i] = weight_7
            elif val_labels[i] == 8:
                validation_cost[i] = weight_8
            elif val_labels[i] == 9:
                validation_cost[i] = weight_9
            elif val_labels[i] == 10:
                validation_cost[i] = weight_10
            elif val_labels[i] == 11:
                validation_cost[i] = weight_11
            elif val_labels[i] == 12:
                validation_cost[i] = weight_12
            elif val_labels[i] == 13:
                validation_cost[i] = weight_13
            elif val_labels[i] == 14:
                validation_cost[i] = weight_14

        # Calculate validation metrics:
        val_acc = accuracy_score(val_labels,val_preds)
        val_acc_weight = accuracy_score(val_labels,val_preds,sample_weight=validation_cost)
        val_prec = precision_score(val_labels,val_preds,average=None)
        val_rec = recall_score(val_labels,val_preds,average=None)
        val_f1 = f1_score(val_labels,val_preds,average=None)
        count+=1

        # Save best results:
        if val_acc_weight > best_acc_weight:
            best_acc_weight = val_acc_weight
            best_results = (count, val_acc, val_acc_weight, val_prec, val_rec, val_f1)

        # Calculate validation metrics:
        print("\nEpoch: ",count)
        print("\t - Validation Accuracy: {:.4f}".format(val_acc))
        print("\t - Validation Accuracy (weighted): {:.4f}\n".format(val_acc_weight))
        print("\t - Validation Precision (class 0): {:.4f}".format(val_prec[0]))
        print("\t - Validation Recall (class 0): {:.4f}".format(val_rec[0]))
        print("\t - Validation F1-score (class 0): {:.4f}\n".format(val_f1[0]))
        print("\t - Validation Precision (class 1): {:.4f}".format(val_prec[1]))
        print("\t - Validation Recall (class 1): {:.4f}".format(val_rec[1]))
        print("\t - Validation F1-score (class 1): {:.4f}\n".format(val_f1[1]))
        print("\t - Validation Precision (class 2): {:.4f}".format(val_prec[2]))
        print("\t - Validation Recall (class 2): {:.4f}".format(val_rec[2]))
        print("\t - Validation F1-score (class 2): {:.4f}\n".format(val_f1[2]))
        print("\t - Validation Precision (class 3): {:.4f}".format(val_prec[3]))
        print("\t - Validation Recall (class 3): {:.4f}".format(val_rec[3]))
        print("\t - Validation F1-score (class 3): {:.4f}\n".format(val_f1[3]))
        print("\t - Validation Precision (class 4): {:.4f}".format(val_prec[4]))
        print("\t - Validation Recall (class 4): {:.4f}".format(val_rec[4]))
        print("\t - Validation F1-score (class 4): {:.4f}\n".format(val_f1[4]))
        print("\t - Validation Precision (class 5): {:.4f}".format(val_prec[5]))
        print("\t - Validation Recall (class 5): {:.4f}".format(val_rec[5]))
        print("\t - Validation F1-score (class 5): {:.4f}\n".format(val_f1[5]))
        print("\t - Validation Precision (class 6): {:.4f}".format(val_prec[6]))
        print("\t - Validation Recall (class 6): {:.4f}".format(val_rec[6]))
        print("\t - Validation F1-score (class 6): {:.4f}\n".format(val_f1[6]))
        print("\t - Validation Precision (class 7): {:.4f}".format(val_prec[7]))
        print("\t - Validation Recall (class 7): {:.4f}".format(val_rec[7]))
        print("\t - Validation F1-score (class 7): {:.4f}\n".format(val_f1[7]))
        print("\t - Validation Precision (class 8): {:.4f}".format(val_prec[8]))
        print("\t - Validation Recall (class 8): {:.4f}".format(val_rec[8]))
        print("\t - Validation F1-score (class 8): {:.4f}\n".format(val_f1[8]))
        print("\t - Validation Precision (class 9): {:.4f}".format(val_prec[9]))
        print("\t - Validation Recall (class 9): {:.4f}".format(val_rec[9]))
        print("\t - Validation F1-score (class 9): {:.4f}\n".format(val_f1[9]))
        print("\t - Validation Precision (class 10): {:.4f}".format(val_prec[10]))
        print("\t - Validation Recall (class 10): {:.4f}".format(val_rec[10]))
        print("\t - Validation F1-score (class 10): {:.4f}\n".format(val_f1[10]))
        print("\t - Validation Precision (class 11): {:.4f}".format(val_prec[11]))
        print("\t - Validation Recall (class 11): {:.4f}".format(val_rec[11]))
        print("\t - Validation F1-score (class 11): {:.4f}\n".format(val_f1[11]))
        print("\t - Validation Precision (class 12): {:.4f}".format(val_prec[12]))
        print("\t - Validation Recall (class 12): {:.4f}".format(val_rec[12]))
        print("\t - Validation F1-score (class 12): {:.4f}\n".format(val_f1[12]))
        print("\t - Validation Precision (class 13): {:.4f}".format(val_prec[13]))
        print("\t - Validation Recall (class 13): {:.4f}".format(val_rec[13]))
        print("\t - Validation F1-score (class 13): {:.4f}\n".format(val_f1[13]))
        print("\t - Validation Precision (class 14): {:.4f}".format(val_prec[14]))
        print("\t - Validation Recall (class 14): {:.4f}".format(val_rec[14]))
        print("\t - Validation F1-score (class 14): {:.4f}\n".format(val_f1[14]))

    # Best results of an epoch:
    print("\nBest results:")
    print("\nEpoch: ",best_results[0])
    print("\t - Validation Accuracy: {:.4f}".format(best_results[1]))
    print("\t - Validation Accuracy (weighted): {:.4f}\n".format(best_results[2]))
    print("\t - Validation Precision (class 0): {:.4f}".format(best_results[3][0]))
    print("\t - Validation Recall (class 0): {:.4f}".format(best_results[4][0]))
    print("\t - Validation F1-score (class 0): {:.4f}\n".format(best_results[5][0]))
    print("\t - Validation Precision (class 1): {:.4f}".format(best_results[3][1]))
    print("\t - Validation Recall (class 1): {:.4f}".format(best_results[4][1]))
    print("\t - Validation F1-score (class 1): {:.4f}\n".format(best_results[5][1]))
    print("\t - Validation Precision (class 2): {:.4f}".format(best_results[3][2]))
    print("\t - Validation Recall (class 2): {:.4f}".format(best_results[4][2]))
    print("\t - Validation F1-score (class 2): {:.4f}\n".format(best_results[5][2]))
    print("\t - Validation Precision (class 3): {:.4f}".format(best_results[3][3]))
    print("\t - Validation Recall (class 3): {:.4f}".format(best_results[4][3]))
    print("\t - Validation F1-score (class 3): {:.4f}\n".format(best_results[5][3]))
    print("\t - Validation Precision (class 4): {:.4f}".format(best_results[3][4]))
    print("\t - Validation Recall (class 4): {:.4f}".format(best_results[4][4]))
    print("\t - Validation F1-score (class 4): {:.4f}\n".format(best_results[5][4]))
    print("\t - Validation Precision (class 5): {:.4f}".format(best_results[3][5]))
    print("\t - Validation Recall (class 5): {:.4f}".format(best_results[4][5]))
    print("\t - Validation F1-score (class 5): {:.4f}\n".format(best_results[5][5]))
    print("\t - Validation Precision (class 6): {:.4f}".format(best_results[3][6]))
    print("\t - Validation Recall (class 6): {:.4f}".format(best_results[4][6]))
    print("\t - Validation F1-score (class 6): {:.4f}\n".format(best_results[5][6]))
    print("\t - Validation Precision (class 7): {:.4f}".format(best_results[3][7]))
    print("\t - Validation Recall (class 7): {:.4f}".format(best_results[4][7]))
    print("\t - Validation F1-score (class 7): {:.4f}\n".format(best_results[5][7]))
    print("\t - Validation Precision (class 8): {:.4f}".format(best_results[3][8]))
    print("\t - Validation Recall (class 8): {:.4f}".format(best_results[4][8]))
    print("\t - Validation F1-score (class 8): {:.4f}\n".format(best_results[5][8]))
    print("\t - Validation Precision (class 9): {:.4f}".format(best_results[3][9]))
    print("\t - Validation Recall (class 9): {:.4f}".format(best_results[4][9]))
    print("\t - Validation F1-score (class 9): {:.4f}\n".format(best_results[5][9]))
    print("\t - Validation Precision (class 10): {:.4f}".format(best_results[3][10]))
    print("\t - Validation Recall (class 10): {:.4f}".format(best_results[4][10]))
    print("\t - Validation F1-score (class 10): {:.4f}\n".format(best_results[5][10]))
    print("\t - Validation Precision (class 11): {:.4f}".format(best_results[3][11]))
    print("\t - Validation Recall (class 11): {:.4f}".format(best_results[4][11]))
    print("\t - Validation F1-score (class 11): {:.4f}\n".format(best_results[5][11]))
    print("\t - Validation Precision (class 12): {:.4f}".format(best_results[3][12]))
    print("\t - Validation Recall (class 12): {:.4f}".format(best_results[4][12]))
    print("\t - Validation F1-score (class 12): {:.4f}\n".format(best_results[5][12]))
    print("\t - Validation Precision (class 13): {:.4f}".format(best_results[3][13]))
    print("\t - Validation Recall (class 13): {:.4f}".format(best_results[4][13]))
    print("\t - Validation F1-score (class 13): {:.4f}\n".format(best_results[5][13]))
    print("\t - Validation Precision (class 14): {:.4f}".format(best_results[3][14]))
    print("\t - Validation Recall (class 14): {:.4f}".format(best_results[4][14]))
    print("\t - Validation F1-score (class 14): {:.4f}\n".format(best_results[5][14]))
    average_results.append(best_results)

# Determine best weighted accuracy:
max_acc_weight = np.max([i[2] for i in average_results])
index_best = [i for i in range(len(average_results)) if average_results[i][2]==max_acc_weight][0]

# Output best and averaged results:
print("\n-------------------------------------------------------")
print("\t - Averaged F1-score (class 0): {:.4f}".format(np.mean([i[5][0] for i in average_results])))
print("\t - Best F1-score (class 0): {:.4f}\n".format(average_results[index_best][5][0]))
print("\t - Averaged F1-score (class 1): {:.4f}".format(np.mean([i[5][1] for i in average_results])))
print("\t - Best F1-score (class 1): {:.4f}\n".format(average_results[index_best][5][1]))
print("\t - Averaged F1-score (class 2): {:.4f}".format(np.mean([i[5][2] for i in average_results])))
print("\t - Best F1-score (class 2): {:.4f}\n".format(average_results[index_best][5][2]))
print("\t - Averaged F1-score (class 3): {:.4f}".format(np.mean([i[5][3] for i in average_results])))
print("\t - Best F1-score (class 3): {:.4f}\n".format(average_results[index_best][5][3]))
print("\t - Averaged F1-score (class 4): {:.4f}".format(np.mean([i[5][4] for i in average_results])))
print("\t - Best F1-score (class 4): {:.4f}\n".format(average_results[index_best][5][4]))
print("\t - Averaged F1-score (class 5): {:.4f}".format(np.mean([i[5][5] for i in average_results])))
print("\t - Best F1-score (class 5): {:.4f}\n".format(average_results[index_best][5][5]))
print("\t - Averaged F1-score (class 6): {:.4f}".format(np.mean([i[5][6] for i in average_results])))
print("\t - Best F1-score (class 6): {:.4f}\n".format(average_results[index_best][5][6]))
print("\t - Averaged F1-score (class 7): {:.4f}".format(np.mean([i[5][7] for i in average_results])))
print("\t - Best F1-score (class 7): {:.4f}\n".format(average_results[index_best][5][7]))
print("\t - Averaged F1-score (class 8): {:.4f}".format(np.mean([i[5][8] for i in average_results])))
print("\t - Best F1-score (class 8): {:.4f}\n".format(average_results[index_best][5][8]))
print("\t - Averaged F1-score (class 9): {:.4f}".format(np.mean([i[5][9] for i in average_results])))
print("\t - Best F1-score (class 9): {:.4f}\n".format(average_results[index_best][5][9]))
print("\t - Averaged F1-score (class 10): {:.4f}".format(np.mean([i[5][10] for i in average_results])))
print("\t - Best F1-score (class 10): {:.4f}\n".format(average_results[index_best][5][10]))
print("\t - Averaged F1-score (class 11): {:.4f}".format(np.mean([i[5][11] for i in average_results])))
print("\t - Best F1-score (class 11): {:.4f}\n".format(average_results[index_best][5][11]))
print("\t - Averaged F1-score (class 12): {:.4f}".format(np.mean([i[5][12] for i in average_results])))
print("\t - Best F1-score (class 12): {:.4f}\n".format(average_results[index_best][5][12]))
print("\t - Averaged F1-score (class 13): {:.4f}".format(np.mean([i[5][13] for i in average_results])))
print("\t - Best F1-score (class 13): {:.4f}\n".format(average_results[index_best][5][13]))
print("\t - Averaged F1-score (class 14): {:.4f}".format(np.mean([i[5][14] for i in average_results])))
print("\t - Best F1-score (class 14): {:.4f}\n\n".format(average_results[index_best][5][14]))

print("\t - Max weighted accuracy: {:.4f}\n\n".format(max_acc_weight))

print("Line to copy to .xls:")
print((str(average_results[index_best][5][0])+'\t'+str(average_results[index_best][5][1])+'\t'+str(average_results[index_best][5][2])+'\t'+str(average_results[index_best][5][3])+'\t'+str(average_results[index_best][5][4])+'\t'+str(average_results[index_best][5][5])+'\t'+str(average_results[index_best][5][6])+'\t'+str(average_results[index_best][5][7])+'\t'+str(average_results[index_best][5][8])+'\t'+str(average_results[index_best][5][9])+'\t'+str(average_results[index_best][5][10])+'\t'+str(average_results[index_best][5][11])+'\t'+str(average_results[index_best][5][12])+'\t'+str(average_results[index_best][5][13])+'\t'+str(average_results[index_best][5][14])+'\t'+str(max_acc_weight)).replace('.',','))


Experiment 1 --------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...


Epoch:  8
	 - Validation Accuracy: 0.6692
	 - Validation Accuracy (weighted): 0.6350

	 - Validation Precision (class 0): 0.6336
	 - Validation Recall (class 0): 0.5570
	 - Validation F1-score (class 0): 0.5929

	 - Validation Precision (class 1): 0.5789
	 - Validation Recall (class 1): 0.9565
	 - Validation F1-score (class 1): 0.7213

	 - Validation Precision (class 2): 0.6129
	 - Validation Recall (class 2): 0.7308
	 - Validation F1-score (class 2): 0.6667

	 - Validation Precision (class 3): 0.8000
	 - Validation Recall (class 3): 0.5263
	 - Validation F1-score (class 3): 0.6349

	 - Validation Precision (class 4): 0.7500
	 - Validation Recall (class 4): 0.5000
	 - Validation F1-score (class 4): 0.6000

	 - Validation Precision (class 5): 0.9000
	 - Validation Recall (class 5): 1.0000
	 - Validation F1-score (class 5): 0.9474

	 - Validation Precision (class 6): 0.9630
	 - Validation Recall (class 6): 0.8966
	 - Validation F1-score (class 6): 0.9286

	 - Validation Precision (class

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Epoch:  90%|█████████ | 9/10 [10:35<01:10, 70.62s/it]


Epoch:  9
	 - Validation Accuracy: 0.7172
	 - Validation Accuracy (weighted): 0.6856

	 - Validation Precision (class 0): 0.7143
	 - Validation Recall (class 0): 0.5369
	 - Validation F1-score (class 0): 0.6130

	 - Validation Precision (class 1): 0.6176
	 - Validation Recall (class 1): 0.9130
	 - Validation F1-score (class 1): 0.7368

	 - Validation Precision (class 2): 0.6562
	 - Validation Recall (class 2): 0.8077
	 - Validation F1-score (class 2): 0.7241

	 - Validation Precision (class 3): 0.7179
	 - Validation Recall (class 3): 0.7368
	 - Validation F1-score (class 3): 0.7273

	 - Validation Precision (class 4): 0.8000
	 - Validation Recall (class 4): 0.6667
	 - Validation F1-score (class 4): 0.7273

	 - Validation Precision (class 5): 0.8182
	 - Validation Recall (class 5): 1.0000
	 - Validation F1-score (class 5): 0.9000

	 - Validation Precision (class 6): 0.9333
	 - Validation Recall (class 6): 0.9655
	 - Validation F1-score (class 6): 0.9492

	 - Validation Precision (class

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Epoch: 100%|██████████| 10/10 [11:46<00:00, 70.65s/it]


Epoch:  10
	 - Validation Accuracy: 0.7273
	 - Validation Accuracy (weighted): 0.6715

	 - Validation Precision (class 0): 0.6481
	 - Validation Recall (class 0): 0.7047
	 - Validation F1-score (class 0): 0.6752

	 - Validation Precision (class 1): 0.8182
	 - Validation Recall (class 1): 0.7826
	 - Validation F1-score (class 1): 0.8000

	 - Validation Precision (class 2): 0.6923
	 - Validation Recall (class 2): 0.6923
	 - Validation F1-score (class 2): 0.6923

	 - Validation Precision (class 3): 0.7000
	 - Validation Recall (class 3): 0.7368
	 - Validation F1-score (class 3): 0.7179

	 - Validation Precision (class 4): 0.7273
	 - Validation Recall (class 4): 0.4444
	 - Validation F1-score (class 4): 0.5517

	 - Validation Precision (class 5): 1.0000
	 - Validation Recall (class 5): 1.0000
	 - Validation F1-score (class 5): 1.0000

	 - Validation Precision (class 6): 0.9630
	 - Validation Recall (class 6): 0.8966
	 - Validation F1-score (class 6): 0.9286

	 - Validation Precision (clas